In [9]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [10]:
dataset_path = "dataset.csv"
df = pd.read_csv(dataset_path)

In [11]:
# Remove negative values from the dataset
for column in df.select_dtypes(include=['int64', 'float64']).columns:
    df = df[df[column] >= 0]

print('Negative values removed from the dataset.')

Negative values removed from the dataset.


In [12]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['month', 'day', 'time', 'humidity', 'tempC']])

In [13]:
X = scaled_features
y = df['fan_speed'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  # Softmax activation for multi-class classification
])

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)


Epoch 1/100
88/88 [==============================] - 1s 2ms/step - loss: 0.2259 - accuracy: 0.8998 - val_loss: 0.2317 - val_accuracy: 0.8890
Epoch 2/100
88/88 [==============================] - 0s 1ms/step - loss: 0.2277 - accuracy: 0.8981 - val_loss: 0.2415 - val_accuracy: 0.8855
Epoch 3/100
88/88 [==============================] - 0s 1ms/step - loss: 0.2245 - accuracy: 0.9011 - val_loss: 0.2359 - val_accuracy: 0.8933
Epoch 4/100
88/88 [==============================] - 0s 1ms/step - loss: 0.2248 - accuracy: 0.8986 - val_loss: 0.2350 - val_accuracy: 0.8926
Epoch 5/100
88/88 [==============================] - 0s 1ms/step - loss: 0.2280 - accuracy: 0.8970 - val_loss: 0.2371 - val_accuracy: 0.8883
Epoch 6/100
88/88 [==============================] - 0s 1ms/step - loss: 0.2242 - accuracy: 0.9020 - val_loss: 0.2444 - val_accuracy: 0.8826
Epoch 7/100
88/88 [==============================] - 0s 1ms/step - loss: 0.2223 - accuracy: 0.9014 - val_loss: 0.2321 - val_accuracy: 0.8919
Epoch 8/100
8

In [37]:
# Convert true classes to one-hot encoding
true_classes_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=6)
predicted_classes = model.predict(X_test)
# Calculate accuracy
accuracy = np.mean(np.argmax(predicted_classes, axis=1) == np.argmax(true_classes_one_hot, axis=1))
print("Test Accuracy (Threshold):", accuracy)


55/55 [==============================] - 0s 724us/step
Test Accuracy (Threshold): 0.9026750142287991


In [38]:
model.save('model.h5')

In [22]:
loaded_model = tf.keras.models.load_model('model.h5')

# Use the loaded model to make predictions on test data
predictions = loaded_model.predict(X_test)

# Create a DataFrame to compare predicted values and actual validation values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions.flatten()})
print(comparison_df)

55/55 [==============================] - 0s 782us/step


ValueError: All arrays must be of the same length